# 回答候補が１つしかない場合のprobability


学習時に重み付けスコア／バイアス値が１件だけ高すぎる結果となった場合（＝回答候補が１件しか存在しない場合）、probabilityが高く計算されてしまうことの裏付けをとります。

## (1) 調査用の環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

learning_dir

'/Users/makmorit/GitHub/donusagi-bot/learning'

In [2]:
import numpy as np
from learning.core.persistance import Persistance
from learning.core.predict.model_not_exists_error import ModelNotExistsError

In [3]:
def load_model_and_vectorizer(bot_id):
    '''
        学習済みのモデルを復元
    '''
    try:
        estimator = Persistance.load_model(bot_id)
        vectorizer = Persistance.load_vectorizer(bot_id)
    except IOError:
        raise ModelNotExistsError()

    return estimator, vectorizer

In [4]:
bot_id = 9  # bot_id = 9はセプテーニ

estimator, vectorizer = load_model_and_vectorizer(bot_id)
print("n_answer=%d, n_feature=%d" % (estimator.coef_.shape[0], estimator.coef_.shape[1]))

n_answer=174, n_feature=1154


### (1-2) feature「やめる」から誘導されやすいクラスを取得

proba 内部処理時における、回答候補クラスの変数値をウォッチするため、回答候補クラスのインデックスをあらかじめ取得しておきます。

In [5]:
def research_preferred_answer_ids(estimator, vectorizer, feature_word, n_top=10):
    '''
        重みテーブルから、featureに対応する列を抽出し、
        重みスコアの降順に、answer_idと共に重みスコアのリストを出力
        デフォルト＝トップの１０件のみ出力
    '''
    feature_index = vectorizer.vocabulary_[feature_word]
    print("research_preferred_answer_ids: feature word=%s (index=%d of %d)" % (
        feature_word, feature_index, estimator.coef_.shape[1]))

    _table_w = []
    for class_index, weight in enumerate(estimator.coef_.T[feature_index]):
        answer_id = estimator.classes_[class_index]
        _table_w.append((class_index, answer_id, weight))
        
    sorted_table_w = sorted(_table_w, key=lambda x:x[2], reverse=True)

    return sorted_table_w[0:n_top]

In [6]:
research_preferred_answer_ids(estimator, vectorizer, 'やめる')

research_preferred_answer_ids: feature word=やめる (index=124 of 1154)


[(114, 4579, 10.754334344795975),
 (107, 4572, 2.2669260980155244),
 (34, 4497, -4.9621819786803401e-05),
 (33, 4496, -5.299633357049133e-05),
 (12, 4445, -0.00013470836509254396),
 (45, 4508, -0.00020345183422650895),
 (98, 4563, -0.00020468957614262404),
 (145, 4610, -0.00021637775059685249),
 (42, 4505, -0.00022534947812052333),
 (10, 4441, -0.0002939213319672891)]

## (2) predict_probaの再現実行

素のままではわからないので、適宜デバッグ・プリントを入れて予測処理を再現実行します。

学習時に計算した重み付けスコア／バイアス値から、ロジスティック回帰値を経て、probability（確率）が算出される過程をデバッグプリントしています。

結果、<b>回答候補が１件しか存在しない場合、probabilityが高く計算されてしまう</b>ことが確認できるかと存じます。

In [7]:
from learning.core.predict.reply import Reply
from learning.tests import helper

questions = ['やめる']
result = Reply(bot_id, helper.learning_parameter(use_similarity_classification=False)).perform(questions, datasource_type='csv')
result

2017/05/11 AM 11:41:06 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/11 AM 11:41:06 ['./fixtures/question_answers/toyotsu_human.csv']
2017/05/11 AM 11:41:06 TextArray#__init__ start
2017/05/11 AM 11:41:06 Reply#perform text_array.separated_sentences: ['やめる']
2017/05/11 AM 11:41:06 TextArray#to_vec start
2017/05/11 AM 11:41:06 TextArray#to_vec end
2017/05/11 AM 11:41:06 Reply#perform features:   (0, 124)	1.0
2017/05/11 AM 11:41:06 question: やめる
2017/05/11 AM 11:41:06 question_feature_count: 1
2017/05/11 AM 11:41:06 predicted results (order by probability desc)
2017/05/11 AM 11:41:06 {'answer_id': 4579.0, 'probability': 0.95788732602549198}
2017/05/11 AM 11:41:06 {'answer_id': 0.0, 'probability': 0.023378873591347195}
2017/05/11 AM 11:41:06 {'answer_id': 4454.0, 'probability': 0.0038913316449180053}
2017/05/11

LinearClassifierMixin#_predict_proba_lr: [1]学習時に算出した重み付けスコア＋バイアスから影響変数を計算
LinearClassifierMixin#_predict_proba_lr: label=4579, prob[114]= -1.16222837053
LinearClassifierMixin#_predict_proba_lr: label=4572, prob[107]= -8.79820586678
LinearClassifierMixin#_predict_proba_lr: label=4497, prob[34]= -19.24736899
LinearClassifierMixin#_predict_proba_lr: label=4496, prob[33]= -18.0150830965
LinearClassifierMixin#_predict_proba_lr: label=4445, prob[12]= -15.6793939248
LinearClassifierMixin#_predict_proba_lr: [2]影響変数からロジスティック回帰値を計算
LinearClassifierMixin#_predict_proba_lr: label=4579, prob[114]= 0.238262614444
LinearClassifierMixin#_predict_proba_lr: label=4572, prob[107]= 0.000150980954365
LinearClassifierMixin#_predict_proba_lr: label=4497, prob[34]= 4.37495766246e-09
LinearClassifierMixin#_predict_proba_lr: label=4496, prob[33]= 1.50019879975e-08
LinearClassifierMixin#_predict_proba_lr: label=4445, prob[12]= 1.55069244689e-07
LinearClassifierMixin#_predict_proba_lr: [3]ロジスティック回帰値を標準化してprobabil

### ご参考：デバッグプリントに使用したロジック

base.py の LinearClassifierMixin#_predict_proba_lr に、下記のようなデバッグプリント処理を追加しています。

In [ ]:
class LinearClassifierMixin(ClassifierMixin):

    def _predict_proba_lr(self, X):
        """Probability estimation for OvR logistic regression.

        Positive class probabilities are computed as
        1. / (1. + np.exp(-self.decision_function(X)));
        multiclass is handled by normalizing that over all classes.
        """

        prob = self.decision_function(X)
        print("LinearClassifierMixin#_predict_proba_lr: [1]学習時に算出した重み付けスコア＋バイアスから影響変数を計算")
        print("LinearClassifierMixin#_predict_proba_lr: label=4579, prob[114]=", prob[0][114])
        print("LinearClassifierMixin#_predict_proba_lr: label=4572, prob[107]=", prob[0][107])
        print("LinearClassifierMixin#_predict_proba_lr: label=4497, prob[34]=", prob[0][34])
        print("LinearClassifierMixin#_predict_proba_lr: label=4496, prob[33]=", prob[0][33])
        print("LinearClassifierMixin#_predict_proba_lr: label=4445, prob[12]=", prob[0][12])

        prob *= -1
        np.exp(prob, prob)
        prob += 1
        np.reciprocal(prob, prob)

        print("LinearClassifierMixin#_predict_proba_lr: [2]影響変数からロジスティック回帰値を計算")
        print("LinearClassifierMixin#_predict_proba_lr: label=4579, prob[114]=", prob[0][114])
        print("LinearClassifierMixin#_predict_proba_lr: label=4572, prob[107]=", prob[0][107])
        print("LinearClassifierMixin#_predict_proba_lr: label=4497, prob[34]=", prob[0][34])
        print("LinearClassifierMixin#_predict_proba_lr: label=4496, prob[33]=", prob[0][33])
        print("LinearClassifierMixin#_predict_proba_lr: label=4445, prob[12]=", prob[0][12])

        if prob.ndim == 1:
            return np.vstack([1 - prob, prob]).T
        else:
            # OvR normalization, like LibLinear's predict_probability
            prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))

            print("LinearClassifierMixin#_predict_proba_lr: [3]ロジスティック回帰値を標準化してprobabilityを計算")
            print("LinearClassifierMixin#_predict_proba_lr: label=4579, prob[114]=", prob[0][114])
            print("LinearClassifierMixin#_predict_proba_lr: label=4572, prob[107]=", prob[0][107])
            print("LinearClassifierMixin#_predict_proba_lr: label=4497, prob[34]=", prob[0][34])
            print("LinearClassifierMixin#_predict_proba_lr: label=4496, prob[33]=", prob[0][33])
            print("LinearClassifierMixin#_predict_proba_lr: label=4445, prob[12]=", prob[0][12])
            return prob
